In [1]:
# here we run through calculate for all ml dataset as set in 

import prepare_ml_dataset
import importlib
importlib.reload(prepare_ml_dataset)

prepare_ml_dataset.prepare_ml_dataset_batch()

/home/jliao/anaconda3/envs/notebook/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-25 17:16:23.602085: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-25 17:16:23.610744: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-25 17:16:23.688679: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-25 17:16:23.758703: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to r

In [ ]:
# This code is for testing the load_ml_dataset.


# import prepare_ml_dataset
# import importlib
# importlib.reload(prepare_ml_dataset)

# raw_feature_names =  ['symh','asyh','ae','asyd'] #['symh','asyh','asyd','ae','f10.7','kp','swp','swn','swv','by','bz']

# # [51767.680, 44428.696, 38130.120, 32724.498, 28085.268, 24103.668, 20686.558, 17753.876, 15236.896, 13076.798, 11222.936, 9631.899, 8266.406, 7094.516, 6088.722, 5225.528, 4484.742
# # , 3848.919, 3303.284, 2834.964, 2433.055, 2088.129, 1792.096, 1538.062, 1319.977, 1132.846, 972.237]
# number_history_arr = [7]
# forecast_arr = ["all"]
# dL01_arr = [True]
# species_arr = ['h']
# energy_arr = ['972237']  

# for number_history in number_history_arr:
#     for forecast in forecast_arr:
#        for dL01 in dL01_arr:
#            for species in species_arr:
#                for energy in energy_arr:
               
#                     prepare_ml_dataset.prepare_ml_dataset(energy, species, recalc = True, plot_data = True, save_data = False
#                                                           , dL01=dL01, forecast = forecast, number_history =number_history, raw_feature_names =  raw_feature_names)


In [ ]:
# All loops together is a standard routine for create all combination of training/validation/test data for a perticular raw_feature_names setting

import importlib
importlib.reload(prepare_ml_dataset)

raw_feature_names =  ['symh','asyh','ae','asyd'] #['symh','asyh','asyd','ae','f10.7','kp','swp','swn','swv','by','bz']

# [51767.680, 44428.696, 38130.120, 32724.498, 28085.268, 24103.668, 20686.558, 17753.876, 15236.896, 13076.798, 11222.936, 9631.899, 8266.406, 7094.516, 6088.722, 5225.528, 4484.742
# , 3848.919, 3303.284, 2834.964, 2433.055, 2088.129, 1792.096, 1538.062, 1319.977, 1132.846, 972.237]
number_history_arr = [7,8]
forecast_arr = ["all", "index","none"]
dL01_arr = [True, False]
species_arr = ['h', 'o']
energy_arr = ['972237', '51767680']  

for number_history in number_history_arr:
    for forecast in forecast_arr:
       for dL01 in dL01_arr:
           for species in species_arr:
               for energy in energy_arr:
               
                    prepare_ml_dataset.load_ml_dataset(energy, species, recalc = True, plot_data = False, save_data = False
                                                          , dL01=dL01, forecast = forecast, number_history =number_history, raw_feature_names =  raw_feature_names)


In [2]:
import prepare_fulldata
import importlib
import initialize_var
import prepare_ml_dataset

importlib.reload(prepare_fulldata)
importlib.reload(prepare_ml_dataset)
importlib.reload(initialize_var)

recalc = False
plot_data = True
save_data = True

raw_feature_names =  ['symh']#,'asyh','ae','asyd'] #['symh','asyh','asyd','ae','f10.7','kp','swp','swn','swv','by','bz']
number_history = 7
energy = '972237'
species = 'h'
forecast = "none"
dL01 = True
test_ts = '2017-01-01'
test_te = '2018-01-01'

dataset_csv, data_settings, directories = initialize_var.initialize_data_var(energy=energy, species=species, raw_feature_names = raw_feature_names, forecast = forecast, number_history = number_history, test_ts=test_ts, test_te=test_te, dL01=dL01)
        
df_data, directories, fulldataset_csv, fulldata_settings = prepare_fulldata.load_fulldata(energy, species, recalc = recalc, raw_feature_names = raw_feature_names, number_history = number_history, save_data = save_data, plot_data = plot_data)
        
df_full = prepare_fulldata.read_probes_data(directories["rawdata_dir"], fulldata_settings)

df_data[[fulldata_settings['doubletime_name']]] = df_full[[fulldata_settings['doubletime_name']]]

index_good = prepare_ml_dataset.get_good_index(df_full, data_settings, fulldata_settings)

if data_settings["forecast"] == "all":
    data_settings["feature_history_names"] = prepare_ml_dataset.remove_features_by_time(fulldata_settings["feature_history_names"], "*_0h")
elif data_settings["forecast"] == "index":
    data_settings["feature_history_names"] = prepare_ml_dataset.remove_index_features_by_time(fulldata_settings["feature_history_names"], "*_0h")
else:
    data_settings["feature_history_names"] = fulldata_settings["feature_history_names"]            

df_data = df_data.loc[index_good,[fulldata_settings['doubletime_name'], fulldata_settings['datetime_name'],data_settings['y_name'], data_settings['log_y_name']]+ fulldata_settings['coor_names']+fulldata_settings['feature_history_names']]

df_full = df_full.loc[index_good, :]

#-----------------------------
# After this line, both df_data and df_full only have good data. no index_good should be used.

#set test set. Here we use one year (2017) of data for test set 
index_train, index_valid, index_test = prepare_ml_dataset.create_ml_indexes(df_data,  fulldata_settings, data_settings["test_ts"], data_settings["test_te"])

# Each round, one can only train one y. If train more than one y, need to  repeat from here
x_train, x_valid, x_test, y_train, y_valid, y_test = prepare_ml_dataset.create_ml_data(df_data, index_train, index_valid, index_test, data_settings["log_y_name"], fulldata_settings["coor_names"], data_settings["feature_history_names"])  

print("shapes of x_train, x_valid, x_test, y_train, y_valid, y_test ")
print(x_train.shape, x_valid.shape, x_test.shape, y_train.shape, y_valid.shape, y_test.shape)

if save_data:
    prepare_ml_dataset.save_df_data(df_full[[fulldata_settings['datetime_name'], data_settings["y_name"]] + fulldata_settings["raw_coor_names"] + fulldata_settings["raw_feature_names"]], index_train, index_valid, index_test, dataset_csv)

    prepare_ml_dataset.save_csv_data(x_train, x_valid, x_test, y_train, y_valid, y_test , dataset_csv)

output_rel05/fulldata/df_hope_log_h_flux_972237
start symh
Reading from output_rel05/fulldata/df_feature_history_scaled_symh.csv
Reading csv data for probe b

NameError: name 'get_good_index' is not defined

In [6]:
if plot_data:
    print("start plot data") 

    prepare_ml_dataset.plot_y_data(df_data[ [ fulldata_settings['datetime_name'],  data_settings["y_name"],data_settings["log_y_name"]]], data_settings["y_name"],data_settings["log_y_name"],  fulldata_settings['datetime_name'], filename["df_y"]+ '_'+ data_settings["log_y_name"])
    
    prepare_ml_dataset.plot_coor_data(df_data[ [fulldata_settings['datetime_name'],fulldata_settings["coor_names"]  ]], fulldata_settings["coor_names"],  fulldata_settings['datetime_name'], fulldataset_csv["df_coor"])
                
    prepare_ml_dataset.plot_feature_data(df_data[ [fulldata_settings['datetime_name'],fulldata_settings["feature_names"] ]], fulldata_settings["feature_names"],  fulldata_settings['datetime_name'], fulldataset_csv["df_coor"])


start plot data
start plot y data
start viewdata
start plot h_flux_972237
start plot log_h_flux_972237


KeyboardInterrupt: 

Error in callback <function _draw_all_if_interactive at 0x7ec019493d80> (for post_execute), with arguments args (),kwargs {}:


KeyboardInterrupt: 

Error in callback <function flush_figures at 0x7ec0173bf380> (for post_execute), with arguments args (),kwargs {}:



KeyboardInterrupt



In [21]:
filename

'output_rel05/ml_data/symh_history7days_dL01True_forecastnone/h_972237_df_y.csv_log_h_flux_972237'

In [20]:
df_y = df_data[ [ fulldata_settings['datetime_name'],  data_settings["y_name"],data_settings["log_y_name"]]]

y_name = data_settings["y_name"]
log_y_name = data_settings["log_y_name"]
datetime_name = fulldata_settings['datetime_name']
filename = dataset_csv["df_y"]+'_'+  data_settings["log_y_name"]

In [25]:
df_y

,DateTime,h_flux_972237,log_h_flux_972237
3866,2013-01-14 10:12:30,147.995544,2.170249
3869,2013-01-14 10:27:30,74.642838,1.872988
3872,2013-01-14 10:42:30,84.740562,1.928091
3874,2013-01-14 10:52:30,72.566246,1.860735
3876,2013-01-14 11:02:30,93.152679,1.969195
...,...,...,...
1313790,2019-03-31 18:32:30,41.259995,1.615529
1313792,2019-03-31 18:42:30,36.021774,1.556565
1313797,2019-03-31 19:07:30,49.405987,1.693780
1313849,2019-03-31 23:27:30,62.343212,1.794789


In [24]:
import plot_functions

plot_functions.view_data(df_y, [y_name,log_y_name], [y_name,log_y_name], df_y[datetime_name].reset_index(drop=True), figname = filename)

start viewdata
start plot h_flux_972237
start plot log_h_flux_972237


KeyboardInterrupt: 

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7ec0e69aea50>>
Traceback (most recent call last):
  File "/home/jliao/anaconda3/envs/notebook/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


Error in callback <function _draw_all_if_interactive at 0x7ec019493d80> (for post_execute), with arguments args (),kwargs {}:


KeyboardInterrupt: 

Error in callback <function flush_figures at 0x7ec0173bf380> (for post_execute), with arguments args (),kwargs {}:


KeyboardInterrupt: 

array(['2013-01-14 10:12:30', '2013-01-14 10:27:30',
       '2013-01-14 10:42:30', ..., '2019-03-31 19:07:30',
       '2019-03-31 23:27:30', '2019-03-31 23:57:30'], dtype=object)

In [26]:
df_full = df_y

varnames = [y_name,log_y_name]
ylabels = [y_name,log_y_name]

time_array = df_y[datetime_name].astype('datetime64[ns]').reset_index(drop=True)


figname = filename


In [ ]:
time_array = 

In [ ]:
import matplotlib.pyplot as plt

print("start viewdata")
nvar = len(varnames)

fig1, ax1 = plt.subplots(nvar,1, constrained_layout = True)
fig1.set_size_inches(8, nvar*2)
ivar = 0
varname = varnames[ivar]
print("start plot " + varname)
ax1[ivar].set_ylabel(ylabels[ivar])

ax1[ivar].scatter(np.array(time_array),df_full[varname],s = 0.1)



start viewdata
start plot h_flux_972237


Error in callback <function _draw_all_if_interactive at 0x7ec019493d80> (for post_execute), with arguments args (),kwargs {}:


KeyboardInterrupt: 

In [ ]:
ivar = 1
varname = varnames[ivar]
print("start plot " + varname)
ax1[ivar].set_ylabel(ylabels[ivar])

In [ ]:


for ivar in range(len(varnames)):

    varname = varnames[ivar]
    
    print("start plot " + varname)
    
    ax1[ivar].scatter(time_array,df_full[varname],s = 0.1)
    ax1[ivar].set_ylabel(ylabels[ivar])

In [ ]:
plt.savefig(figname + ".png", format = "png", dpi = 300)
